# Rust Futures Explained

In [2]:
:sccache 1
:dep futures = "0.3.*"
:dep tokio = {version = "0.3.3", features = ["full"] }
:dep env_logger = "0.8.3"
:dep log = "0.4.*"
:dep chrono = "0.4.19"

sccache: true


In [3]:
// Arbitrary logger just to show output with timestamps to illustrate timing behavior of futures
use std::io::Write;
use chrono::Local;
use env_logger::Builder;
use log::LevelFilter;

fn config_logger() {
    Builder::new()
        .format(|buf, record| {
            writeln!(buf,
                "{} [{}] - {}",
                Local::now().format("%H:%M:%S%.3f"),
                record.level(),
                record.args()
            )
        })
        .filter(None, LevelFilter::Debug)
        .init();
}


In [4]:
use futures::prelude::*;
use tokio::prelude::*;
config_logger();

## Intro

Rust **Futures** are pending or completed (like any other futures).
Rust **Futures** trait requires a poll function to be implemented.

```
trait SimpleFuture {
    type Output;
    fn poll(&mut self, wake: fn()) -> Poll<Self::Output>;
}

enum Poll<T> {
    Ready(T),
    Pending,
}
```
From:
https://rust-lang.github.io/async-book/02_execution/02_future.html

Futures can be advanced by calling the poll function, which will drive the future as far towards completion as possible. If the future completes, it returns Poll::Ready(result). If the future is not able to complete yet, it returns Poll::Pending and arranges for the wake() function to be called when the Future is ready to make more progress. When wake() is called, the executor driving the Future will call poll again so that the Future can make more progress.

In [ ]:
config_logger();  // needs to be called every block again
// Note: The return type with impl Future... is a short hand for 
// the type that is returned

// This future immideately completes. i.e. it advances directly to comletion
fn simple_ok_future() -> impl Future<Output = Result<String, ()>> {
    use futures::Future;
    use futures::future::ok;
    
    // Here I specify the type of the error as (); otherwise the compiler can't infer it
    let future = ok::<_, ()>(String::from("hello"));
    log::debug!("future: {:?}", future);
    future
}
let mut rt = tokio::runtime::Runtime::new().unwrap();
let result = rt.block_on(simple_ok_future());  // we need to poll a future or block on it.
log::debug!("future: {:?}", result); 


22:38:11.159 [DEBUG] - future: Ready(Some(Ok("hello")))
22:38:11.159 [DEBUG] - future: Ok("hello")


## future::lazy

https://docs.rs/futures/0.2.1/futures/future/fn.lazy.html
https://stackoverflow.com/questions/56544469/what-is-futuresfuturelazy-for

Like a plain closure, it's used to prevent code from being eagerly evaluated. In synchronous terms, it's the difference between calling a function and calling the closure that the function returned

In [5]:
config_logger();
fn sync() -> impl FnOnce() {
    log::info!("This is run when the function is called");
    
    || log::info!("This is run when the return value is called")
}

fn main() {
    let a = sync();
    log::info!("Called the function");
    a();
}
main()

22:42:40.314 [INFO] - This is run when the function is called
22:42:40.315 [INFO] - Called the function
22:42:40.315 [INFO] - This is run when the return value is called


()

In [ ]:
config_logger();
let x = 42;
let mut rt = tokio::runtime::Runtime::new().unwrap();
// note: a future::lazy cannot be assigned to a var in notebook
// the rust jupyter kernel cannot persist this data type
// so we put it into the block_on as parameter directly
let result = rt.block_on(future::lazy(|_| {
    log::debug!("in async block, x => {}", x);
    x
}));
log::debug!("Result: {:?}", result); 

22:58:17.880 [DEBUG] - in async_capture, x => 42
22:58:17.880 [DEBUG] - Result: 42


## Summary

* future::lazy like eval of a closure (a Lazy struct that implments a Future)
* whereby the closure itself is returned by a function (that is the future::lazy call

# Tokyo Runtime

https://levelup.gitconnected.com/demystifying-closures-futures-and-async-await-in-rust-part-2-futures-abe95ab332a2

* enter - quick and dirty
* spawn - put in more
* block_on - wait/ block until the future completes (might block forever?